In [229]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, date_format, expr, floor, to_date,date_sub,trunc, month
from pyspark.sql.types import IntegerType
from datetime import date
import pyspark.sql.functions as F
from dateutil.relativedelta import relativedelta
import datetime
import time
from pyspark.sql import Window

spark = SparkSession.builder.appName('Ascend')\
                            .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")\
                            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")\
                            .config("spark.hadoop.fs.s3a.access.key", "admin")\
                            .config("spark.hadoop.fs.s3a.secret.key", "password")\
                            .config("spark.hadoop.fs.s3a.path.style.access", "true")\
                            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
                            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
                            .config("spark.sql.adaptive.enabled", "true")\
                            .config("spark.sql.adaptive.skewJoin.enabled", "true")\
                            .config("spark.sql.adaptive.localShuffleReader.enabled", "true")\
                            .config("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")\
                            .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

25/07/09 18:14:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [213]:
%%sql
show tables in ascenddb2;

25/07/09 18:09:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace,tableName,isTemporary


In [9]:
%%sql
create database if not exists ascenddb2;

++
||
++
++

In [214]:
%%sql
create table if not exists ascenddb2.dpd_data (
    CONS_ACCT_KEY INT,
    ACCT_DT DATE,
    DPD INT
    )
USING ICEBERG
PARTITIONED BY (months(ACCT_DT));

++
||
++
++

In [215]:
%%sql
SELECT * from ascenddb2.dpd_data;

CONS_ACCT_KEY,ACCT_DT,DPD


In [210]:
%%sql
DROP TABLE ascenddb2.dpd_data;

++
||
++
++

In [216]:
%%sql
DESCRIBE ascenddb2.dpd_data;

col_name,data_type,comment
CONS_ACCT_KEY,int,None
ACCT_DT,date,None
DPD,int,None
,,
# Partitioning,,
Part 0,months(ACCT_DT),


### Creation of Dummy Data For Processing

In [217]:
%%time
# Parameters
num_accounts = 50000
# months_needed = 335  # e.g., 10 years
months_needed = 240  # e.g., 10 years
total_records = num_accounts * months_needed

# Step 1: Create DataFrame for all combinations
df = spark.range(0, total_records)

# Step 2: Generate account keys and month indices
df = df.withColumn("CONS_ACCT_KEY", (col("id") / months_needed).cast("int") + 1) \
       .withColumn("month_index", (col("id") % months_needed).cast("int"))

# Step 3: Generate date for each month (15th of month starting from Jan 2025)
start_date = datetime.date(1998, 1, 15)
dates = [start_date + relativedelta(months=i) for i in range(months_needed)]
date_map = spark.createDataFrame([(i, d.strftime("%m/%d/%Y")) for i, d in enumerate(dates)],
                                 ["month_index", "ACCT_DT"])

# Step 4: Join to get ACCT_DT
df = df.join(date_map, on="month_index", how="left")

# Step 5: Add DPD (random or deterministic)
df = df.withColumn("DPD", ((col("CONS_ACCT_KEY") + col("month_index")) % 100 + 1))

# Final selection
df = df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD")

# ✅ Verify uniqueness
print("Expected Unique Keys:", df.count())
# print("Distinct (CONS_acct_KEY, ACCT_DT):", df.select("CONS_acct_KEY", "ACCT_DT").distinct().count())

print(f"Existing Partitions: {df.rdd.getNumPartitions()}")

# df = df.repartition(6)
df = df.withColumn("ACCT_DT",to_date(col("ACCT_DT"), "MM/dd/yyyy"))

df.printSchema()

# print(f"New Partitions: {stage_df.rdd.getNumPartitions()}")
# ✅ Save to Parquet
df.write\
.format("iceberg")\
.mode("overwrite")\
.option("compression", "snappy")\
.saveAsTable("ascenddb2.dpd_data")

df.explain()
print("Data generated and saved to dpd_data")


Expected Unique Keys: 12000000


Existing Partitions: 6
root
 |-- CONS_ACCT_KEY: integer (nullable = true)
 |-- ACCT_DT: date (nullable = true)
 |-- DPD: integer (nullable = true)



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [CONS_ACCT_KEY#11765, cast(gettimestamp(ACCT_DT#11773, MM/dd/yyyy, TimestampType, Some(Etc/UTC), false) as date) AS ACCT_DT#11797, (((CONS_ACCT_KEY#11765 + month_index#11768) % 100) + 1) AS DPD#11780]
   +- SortMergeJoin [cast(month_index#11768 as bigint)], [month_index#11772L], LeftOuter
      :- Sort [cast(month_index#11768 as bigint) ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(cast(month_index#11768 as bigint), 200), ENSURE_REQUIREMENTS, [plan_id=33924]
      :     +- Project [(cast((cast(id#11763L as double) / 240.0) as int) + 1) AS CONS_ACCT_KEY#11765, cast((id#11763L % 240) as int) AS month_index#11768]
      :        +- Range (0, 12000000, step=1, splits=6)
      +- Sort [month_index#11772L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(month_index#11772L, 200), ENSURE_REQUIREMENTS, [plan_id=33925]
            +- Filter isnotnull(month_index#11772L)
               +- Scan Ex

In [218]:
spark.sql("SELECT committed_at, snapshot_id, operation FROM ascenddb2.dpd_data.snapshots").show(truncate=False)

+-----------------------+-------------------+---------+
|committed_at           |snapshot_id        |operation|
+-----------------------+-------------------+---------+
|2025-07-09 18:11:13.717|7062837905704210678|append   |
+-----------------------+-------------------+---------+



In [219]:
from datetime import datetime
now_str = datetime.now()
spark.sql(f"CALL demo.system.expire_snapshots(table => 'ascenddb2.dpd_data', older_than => TIMESTAMP '{now_str}', retain_last => 1)").show()

+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|deleted_data_files_count|deleted_position_delete_files_count|deleted_equality_delete_files_count|deleted_manifest_files_count|deleted_manifest_lists_count|deleted_statistics_files_count|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|                       0|                                  0|                                  0|                           0|                           0|                             0|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+



In [220]:
%%sql
select count(1) from ascenddb2.dpd_data.files;

count(1)
6


### Scenario Creation

In [221]:
%%sql
with data as (select * from ascenddb2.dpd_data where cons_acct_key ==336
union all
select * from ascenddb2.dpd_data where cons_acct_key ==337)
select * from data order by cons_acct_key,acct_dt limit 10;

CONS_ACCT_KEY,ACCT_DT,DPD
336,1998-01-15,37
336,1998-02-15,38
336,1998-03-15,39
336,1998-04-15,40
336,1998-05-15,41
336,1998-06-15,42
336,1998-07-15,43
336,1998-08-15,44
336,1998-09-15,45
336,1998-10-15,46


In [222]:
%%sql
DELETE from ascenddb2.dpd_data where CONS_ACCT_KEY == 336 AND ACCT_DT = '1998-04-15'

++
||
++
++

In [223]:
queries = [
    "DELETE FROM ascenddb2.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '1998-01-15'",
    "DELETE FROM ascenddb2.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '1998-04-15'",
    "DELETE FROM ascenddb2.dpd_data WHERE CONS_ACCT_KEY = 337 AND ACCT_DT = '1998-05-15'",
]

for query in queries:
    spark.sql(query)

In [226]:
%%sql
with data as (select * from ascenddb2.dpd_data where cons_acct_key ==336
union all
select * from ascenddb2.dpd_data where cons_acct_key ==337)
select * from data order by cons_acct_key,acct_dt limit 10;

CONS_ACCT_KEY,ACCT_DT,DPD
336,1998-01-15,37
336,1998-02-15,38
336,1998-03-15,39
336,1998-05-15,41
336,1998-06-15,42
336,1998-07-15,43
336,1998-08-15,44
336,1998-09-15,45
336,1998-10-15,46
336,1998-11-15,47


### Start Incremental Processing

In [163]:
%%sql
DROP TABLE ascenddb2.dpd_summary
-- DROP TABLE ascenddb2.latest_dpd_summary

++
||
++
++

In [227]:
%%sql

CREATE TABLE ascenddb2.dpd_summary(
    CONS_ACCT_KEY INT,
    ACCT_DT DATE, 
    DPD INT,
    DPD_GRID STRING
    )
USING iceberg
PARTITIONED BY (months(ACCT_DT));

++
||
++
++

#### Process and Generate the DPD Data with latest_dpd_summary

In [228]:
%%time
# Define month range
start_month = datetime.date(1998, 1, 1)
end_month = datetime.date(1998, 1, 1)  # Adjust end date as needed

month = start_month
result_table = "ascenddb2.dpd_summary"

while month <= end_month:
    next_month = (month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
    # print(f"Next Month: {next_month}")
    prev_month = (month.replace(day=1) - datetime.timedelta(days=1)).replace(day=1)
    # print(f"Previous Month: {prev_month}")
    
    print(f"Processing month: {month} ...")
    
    # Load current month data
    current_df = spark.sql(f"""
        SELECT CONS_ACCT_KEY, ACCT_DT, DPD
        FROM ascenddb2.dpd_data
        WHERE 
        -- CONS_ACCT_KEY in (336,337)  AND 
        ACCT_DT >= DATE '{month}' AND ACCT_DT < DATE '{next_month}'
    """)
    
    # Load last month DPD strings
    try:
        prev_df = spark.sql(f"""
                            SELECT CONS_ACCT_KEY, ACCT_DT as PREV_ACCT_DT, DPD, DPD_GRID
                            FROM {result_table}
                            WHERE 
                            -- CONS_ACCT_KEY in (336,337)  AND 
                            -- ACCT_DT >= DATE '{prev_month}' AND 
                            ACCT_DT < DATE '{month}'
                        """)
    except:
        prev_df = None  # First month case

    if prev_df.count() > 0:
        # Get latest previous record per CONS_ACCT_KEY
        window_spec = (
            Window.partitionBy("CONS_ACCT_KEY")
                  .orderBy(F.col("PREV_ACCT_DT").desc())
        )
        latest_prev = prev_df.withColumn("rn", F.row_number().over(window_spec)).filter("rn = 1").drop("rn")

        # Join with current_df
        merged_df = current_df.join(latest_prev.select("CONS_ACCT_KEY","PREV_ACCT_DT", "DPD_GRID"), on="CONS_ACCT_KEY", how="left")
        merged_df = merged_df.withColumn(
            "MONTH_DIFF",
            (F.month("ACCT_DT") - F.month("PREV_ACCT_DT")) +
            (F.year("ACCT_DT") - F.year("PREV_ACCT_DT")) * 12
        )

        merged_df = merged_df.withColumn(
            "FILLER_ARRAY",
            F.when(F.col("MONTH_DIFF") > 1,
                   F.expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')"))
            .otherwise(F.array())
        )

        # merged_df.show(truncate=False)
        
        merged_df = merged_df.withColumn(
            "Merged_DPD_Array",
            F.concat(
                F.array(F.col("DPD")),
                F.col("FILLER_ARRAY"),
                F.when(F.col("DPD_GRID").isNotNull(), F.split(F.col("DPD_GRID"), "~"))
                 .otherwise(F.array())
            )
        )
    else:
        # No previous data at all
        merged_df = current_df.withColumn("Merged_DPD_Array", F.array(F.col("DPD")))

    # merged_df.show(truncate=False)
    # Pad with "?" & Keep last 36 months
    max_months = 36
    merged_df = merged_df.withColumn(
        "DPD_Array_Trimmed",
        F.when(
            F.size("Merged_DPD_Array") >= max_months,
            F.slice("Merged_DPD_Array", 1, max_months)
        ).otherwise(
            F.concat(
                F.col("Merged_DPD_Array"),F.array([F.lit("?") for _ in range(max_months - 1)])
            )
        )
    ).withColumn(
        "DPD_GRID",
        F.concat_ws("~", "DPD_Array_Trimmed")
    )

    # merged_df.show(truncate=False)
    
    # Select final columns
    final_df = merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD", "DPD_GRID")
    
    # Save for next iteration & final output
    # final_df.write.mode("overwrite").saveAsTable(result_table)
    final_df.show(truncate=False)
    final_df.write\
        .format("iceberg")\
        .mode("append")\
        .saveAsTable("ascenddb2.dpd_summary")
    
    # Next month
    month = next_month

final_df.explain()
print("✅ Completed incremental DPD string generation!")

TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object

In [115]:
%%sql
select * from ascenddb2.dpd_summary where cons_acct_key = 336 and ACCT_DT in ( '1998-01-15','1998-02-15')
union all 
select * from ascenddb2.dpd_summary where cons_acct_key = 337 and ACCT_DT = '1998-02-15';

CONS_ACCT_KEY,ACCT_DT,DPD,DPD_GRID
336,1998-01-15,37,37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-02-15,38,38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
337,1998-02-15,39,39~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?


In [137]:
%%sql
select MAX(ACCT_DT) from ascenddb2.dpd_summary;

max(ACCT_DT)
1998-04-15


In [141]:
%%sql
with data as (
select * from ascenddb2.dpd_summary where cons_acct_key = 336 
union all 
select * from ascenddb2.dpd_summary where cons_acct_key = 337)
select * from data order by cons_acct_key, acct_dt

CONS_ACCT_KEY,ACCT_DT,DPD,DPD_GRID
336,1998-01-15,37,37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-02-15,38,38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-03-15,39,39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-05-15,41,41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-06-15,42,42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
337,1998-02-15,39,39~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
337,1998-03-15,40,40~39~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
337,1998-06-15,43,43~?~?~40~39~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?


In [208]:
%%sql
DROP TABLE ascenddb2.latest_dpd_summary;

++
||
++
++

In [209]:
%%sql
select count(*) as count,"dpd_summary" as table_name from ascenddb2.dpd_summary
union all
select count(*) as count,"dpd_data" as table_name  from ascenddb2.dpd_data
union all
select count(*) as count,"latest_dpd_summary" as table_name  from ascenddb2.latest_dpd_summary

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `ascenddb2`.`dpd_summary` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 58;
'Union false, false
:- 'Union false, false
:  :- 'Aggregate [count(1) AS count#11709L, dpd_summary AS table_name#11710]
:  :  +- 'UnresolvedRelation [ascenddb2, dpd_summary], [], false
:  +- Aggregate [count(1) AS count#11711L, dpd_data AS table_name#11712]
:     +- SubqueryAlias demo.ascenddb2.dpd_data
:        +- RelationV2[CONS_ACCT_KEY#11715, ACCT_DT#11716, DPD#11717] demo.ascenddb2.dpd_data demo.ascenddb2.dpd_data
+- 'Aggregate [count(1) AS count#11713L, latest_dpd_summary AS table_name#11714]
   +- 'UnresolvedRelation [ascenddb2, latest_dpd_summary], [], false


### Generate the lastest_dpd_summary_table

In [231]:
from pyspark.sql import functions as F
from pyspark.sql import Window

# Read full dpd_summary history (yes, one-time full scan)
dpd_summary_df = spark.read.table("ascenddb2.dpd_summary")

# Window to pick latest ACCT_DT per account
window_spec = Window.partitionBy("CONS_ACCT_KEY").orderBy(F.col("ACCT_DT").desc())

# Pick latest record per account
latest_dpd = dpd_summary_df.withColumn("rn", F.row_number().over(window_spec)) \
    .filter("rn = 1") \
    .select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")

# Save it as initial latest_dpd_summary
latest_dpd.write.format("iceberg").mode("overwrite").saveAsTable("ascenddb2.latest_dpd_summary")


In [232]:
%%sql
 select count(1) from ascenddb2.latest_dpd_summary

25/07/09 18:16:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


count(1)
0


#### Process latest_dpd_summary

In [234]:
%%time
# Define month range
start_month = datetime.date(1998, 1, 1)
end_month = datetime.date(2018, 1, 1)  # Adjust end date as needed

month = start_month
result_table = "ascenddb2.dpd_summary"

while month <= end_month:
    next_month = (month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1)
    # print(f"Next Month: {next_month}")
    
    print(f"Processing month: {month} ...")

    # Load current month data
    current_df = spark.sql(f"""
        SELECT CONS_ACCT_KEY, ACCT_DT, DPD
        FROM ascenddb2.dpd_data
        WHERE ACCT_DT >= DATE '{month}' AND ACCT_DT < DATE '{next_month}'
    """)
    
    # Load only latest record per account (tiny table)
    latest_prev = spark.read.table("ascenddb2.latest_dpd_summary").withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
    
    # Join (no longer need to scan historical months)
    merged_df = current_df.join(latest_prev, on="CONS_ACCT_KEY", how="left")
    
    # Continue with DPD_GRID generation as before
    merged_df = merged_df.withColumn(
        "MONTH_DIFF",
        (F.month("ACCT_DT") - F.month("ACCT_DT_prev")) +
        (F.year("ACCT_DT") - F.year("ACCT_DT_prev")) * 12
    ).withColumn(
        "FILLER_ARRAY",
        F.when(F.col("MONTH_DIFF") > 1, F.expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')"))
        .otherwise(F.array())
    ).withColumn(
        "Merged_DPD_Array",
        F.concat(
            F.array(F.col("DPD")),
            F.col("FILLER_ARRAY"),
            F.when(F.col("DPD_GRID").isNotNull(), F.split(F.col("DPD_GRID"), "~")).otherwise(F.array())
        )
    )
    
    # Pad & trim as usual
    merged_df = merged_df.withColumn(
        "DPD_Array_Trimmed",
        F.when(
            F.size("Merged_DPD_Array") >= 36,
            F.slice("Merged_DPD_Array", 1, 36)
        ).otherwise(
            F.concat(F.col("Merged_DPD_Array"),
                     F.array([F.lit("?") for _ in range(35)])
            )
        )
    ).withColumn(
        "DPD_GRID",
        F.concat_ws("~", "DPD_Array_Trimmed")
    )
    
    # Save historical output (same as before)
    merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD", "DPD_GRID") \
        .write.format("iceberg").mode("append").saveAsTable("ascenddb2.dpd_summary")

    # Load current month processed data
    current_month_df = merged_df.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")

    # Revert name of ACCT_DT_prev
    latest_prev = spark.read.table("ascenddb2.latest_dpd_summary")
    
    # Merge: Take latest between old and new per account
    merged_latest = latest_prev.union(current_month_df) \
        .withColumn("rn", F.row_number().over(
            Window.partitionBy("CONS_ACCT_KEY").orderBy(F.col("ACCT_DT").desc())
        )) \
        .filter("rn = 1") \
        .drop("rn")
    
    # Save merged latest back
    merged_latest.write.mode("overwrite").saveAsTable("ascenddb2.latest_dpd_summary")
    
    month = next_month

Processing month: 1998-01-01 ...


Processing month: 1998-02-01 ...


Processing month: 1998-03-01 ...


Processing month: 1998-04-01 ...


Processing month: 1998-05-01 ...


...

Processing month: 2017-09-01 ...


Processing month: 2017-10-01 ...


Processing month: 2017-11-01 ...


Processing month: 2017-12-01 ...


Processing month: 2018-01-01 ...
CPU times: user 17.6 s, sys: 8.09 s, total: 25.7 s
Wall time: 18min 30s


In [236]:
%%sql
select count(*) as count,"dpd_summary" as table_name from ascenddb2.dpd_summary
union all
select count(*) as count,"dpd_data" as table_name  from ascenddb2.dpd_data
union all
select count(*) as count,"latest_dpd_summary" as table_name  from ascenddb2.latest_dpd_summary

count,table_name
11999996,dpd_summary
11999996,dpd_data
50000,latest_dpd_summary


In [237]:
%%sql
with data as (
select cons_acct_key,acct_dt,dpd_grid from ascenddb2.dpd_summary where cons_acct_key in (336,337)
union all
select * from ascenddb2.latest_dpd_summary where cons_acct_key in (336,337))
select * from data order by cons_acct_key,acct_dt

cons_acct_key,acct_dt,dpd_grid
336,1998-01-15,37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-02-15,38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-03-15,39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-05-15,41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-06-15,42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-07-15,43~42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-08-15,44~43~42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-09-15,45~44~43~42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-10-15,46~45~44~43~42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
336,1998-11-15,47~46~45~44~43~42~41~?~39~38~37~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?~?
